<h2>Import all required libraries</h2>

In [1]:

import pandas as pd
import numpy as np
import pickle

from sqlalchemy import create_engine

from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score as R2

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.inspection import permutation_importance

from sklearn import linear_model

from sklearn.preprocessing import MinMaxScaler


import warnings
warnings.filterwarnings("ignore")


<h2><center>Data Preparation</center></h2>

In [2]:
# SQLAlchemy connectable
cnx = create_engine('sqlite:///techtest.db').connect()
  
# table named 'contacts' will be returned as a dataframe.
ad_performance = pd.read_sql_table('ad_performance', cnx)
tag = pd.read_sql_table('tag', cnx)


In [3]:
tag=tag[tag.Confidence>0.9]
def timeExtension(end,start):
    end=pd.Timestamp(end)
    start=pd.Timestamp(start)
    diff=end-start
    return diff.seconds
tag['Exposition']=tag.apply(lambda x: timeExtension(x['End_timestamp'],x['Start_timestamp']),axis=1)

In [4]:
orignalData=pd.merge(
                tag,
                ad_performance,
                how="left",
                on=['Ad_id','Asset_id'],
                suffixes=("_tag", "_perf"),
                copy=True,
                indicator=False,
                validate=None)
orignalData.fillna(0,inplace=True)

<h2><center>Prediction</center></h2>


In [71]:
def getWeek(date):
    return str(date.week) 
orignalData['Week']=orignalData.apply(lambda x:getWeek(x['Date_captured']),axis=1)


def getFrecuency(Asset_type,Exposition):
        if Asset_type=='video' and Exposition>0:
            return Exposition
        else:
            return 1
orignalData['Frecuency']=orignalData.apply(lambda x: getFrecuency(x['Asset_type'],x['Exposition']),axis=1)

### Taking into account the main goals of this excercise, I will mainly leverage Tag_name and Tag_type to build the model, following my main points on the remaining predictors

* Ad_id: not useful information, it is the id of each element in the tables
* Asset_id: this something important if we were to measure performance of each asset, however, here we are trying to explain performance of elements present across all assets. An alternative approach would be to use this as a predictor to understand which asset perform better for each performance metric

* Confidence: only used to filter out elements with confidence below the indicated threshold 0.9

* Date_captured: there is definitely relevant information here, different dates might play a massive role on cpc or ctr and vvr, this could be used to understand what time is better to display certain elements within our assets. However, for the scope of this analysis I will exclude this categorical variable from the model. This decision is  based on  the fact that the product we are trying to sell (mayonnaise) has no strong seasonality and therefore changes in CPC/CTR across time should not be attribute to changes in demand. For the scope of the available data, changes in performance across time should be related to the use of certain assets (that work better) during that time. Under these assumptions time is strongly correlated to tag_name and tag_type and should not be used

* Market: in more detailed analysis this could be used to understand how each element of different assets perform in different regions, for the scope of this analysis I will focus on understand this concept at a global level

* Start_timestamp- End_timestamp : only useful on videos as an indirect measure of exposition time, used to compute frequency of usage and not as an independent predictor
* Start_frame-End_frame : only useful on videos as an indirect measure of exposition time (noticed that we could use this metric in combination with the previous one to approximate FPS, a quality measure of our asset)


In [6]:
orignalData['Tag_type_Tag_name']=orignalData['Tag_type']+'|'+orignalData['Tag_name']
categorical_predictors=['Tag_type_Tag_name']

dummies=list()
for predictor in categorical_predictors:
    dummies.append(pd.get_dummies(orignalData[predictor]))
dummies=pd.concat(dummies,axis=1)
data=pd.concat([orignalData.drop(columns=categorical_predictors),dummies],axis=1)

In [ ]:
notUsefulPredictors=['Ad_id', 'Asset_id', 'Tag_name', 'Tag_type', 'Confidence','Asset_type',
                    'Start_timestamp', 'End_timestamp', 'Start_frame', 'End_frame',
                    'Market', 'Date_captured', 'Creative_link','Week','Exposition','Frecuency',
                    'CTR', 'CPC', 'VVR']

# usefulPrediction-->'Tag_type_Tag_name'

varColums=[c for c in data.columns if c not in notUsefulPredictors]

# varColums


### Relative Importance
* Using CPC as independet variable determine the most relevant features using a non parametric method , in this case it would be GBM
* Using CTR as independet variable determine the most relevant features using a non parametric method , in this case it would be GBM

* Leverage these predefined relevant features as the main predictors for a linear model 
* Take linear model coefficints from these elements to understand how these features are influecing each performance metric 


In [8]:
def runRelativeImportance(indpendentVariable,modelreRun,NetreRun,importanceFilter=0.8):
    X=data.loc[:,varColums]
    y=data.loc[:,indpendentVariable]

    X_train,X_valid,y_train,y_valid=train_test_split(X,y,test_size=0.2,random_state=0)

    def trainModel(learning_rate,n_estimators):
        gbr=GradientBoostingRegressor(learning_rate=learning_rate,
                                    n_estimators=n_estimators)
        gbr.fit(X_train,y_train)
        y_train_predict=gbr.predict(X_train)
        rsme_train=MSE(y_train,y_train_predict)**0.5
        y_valid_predict=gbr.predict(X_valid)
        rsme_valid=MSE(y_valid,y_valid_predict)**0.5
        print(f'{learning_rate} {n_estimators} {rsme_train} {rsme_valid}')
        return rsme_train,rsme_valid

    ############ Determine best parametrs #####################

    if NetreRun:
        nitSize=10
        parameterNet=pd.DataFrame(columns=['learning_rate','n_estimators','rsme_train','rsme_valid'], index=range(nitSize))
        parameterNet['learning_rate']=[0.51-0.05*i for i in range(nitSize)]
        parameterNet['n_estimators']=[100+30*i for i in range(nitSize)]
        rsme=parameterNet.apply(lambda x:trainModel(x['learning_rate'],x['n_estimators']),axis=1)
        parameterNet['rsme_train']=[rsme[row][0] for row in rsme.index]
        parameterNet['rsme_valid']=[rsme[row][1] for row in rsme.index]
        with open(f'parameterNet{indpendentVariable}.pkl', 'wb') as file:
            pickle.dump(parameterNet, file)
    else:
        with open(f'parameterNet{indpendentVariable}.pkl', 'rb') as file:
            parameterNet = pickle.load(file)

    if modelreRun:
        bestParameter=parameterNet[parameterNet.rsme_valid==parameterNet.rsme_valid.min()].reset_index()
        ############ Train the model #####################
        gbr=GradientBoostingRegressor(learning_rate=bestParameter.learning_rate.loc[0],
                                    n_estimators=bestParameter.n_estimators.loc[0],
                                    verbose=1
                                    )
        
        def importantFeatures(model,importanceFilter):
            ################### Determine importance of each feature ###########################
            relativeImportance=pd.DataFrame(columns=['feature','importance'],index=range(len(model.feature_names_in_)))
            relativeImportance['feature']=model.feature_names_in_
            relativeImportance['importance']=model.feature_importances_
            relativeImportance=relativeImportance.sort_values(by='importance',ascending=False).reset_index()
            relativeImportance.drop(columns=['index'],inplace=True)
            relativeImportance['CumSum']=relativeImportance.importance.cumsum()
            #taking the most important features to reduce model complexity
            importantFeatures=relativeImportance[relativeImportance.CumSum<=importanceFilter]
            return importantFeatures

        # gbr.fit(X_train,y_train)
        # importantFeatures=importantFeatures(gbr,importanceFilter)

                
        #retrain the whole model
        gbr.fit(X,y)
        importantFeatures=importantFeatures(gbr,importanceFilter)

        #train model only using important features
        gbr.fit(X_train[importantFeatures.feature],y_train)
        y_predict=gbr.predict(X_valid[importantFeatures.feature])
        r2_gbm=R2(y_valid,y_predict)
        print(f"{indpendentVariable} GBM r2 {r2_gbm}")
       
        
        with open(f'gbr{indpendentVariable}.pkl', 'wb') as file:
            pickle.dump(gbr, file)
        with open(f'r2_gbm{indpendentVariable}.pkl', 'wb') as file:
            pickle.dump(r2_gbm, file)
        with open(f'importantFeatures{indpendentVariable}.pkl', 'wb') as file:
            pickle.dump(importantFeatures, file)
    else:
        with open(f'gbr{indpendentVariable}.pkl', 'rb') as file:
            gbr = pickle.load(file)
        with open(f'r2_gbm{indpendentVariable}.pkl', 'rb') as file:
            r2_gbm = pickle.load(file)
        with open(f'importantFeatures{indpendentVariable}.pkl', 'rb') as file:
            importantFeatures = pickle.load(file)
        
        print(f"{indpendentVariable} GBM r2 {r2_gbm}")

    model_train =  linear_model.LinearRegression(normalize=True)
    model_train.fit(X_train[importantFeatures.feature],y_train)
    y_predict=model_train.predict(X_valid[importantFeatures.feature])

    print(f"{indpendentVariable} MLR r2 {R2(y_valid,y_predict)}")

    model_final =  linear_model.LinearRegression(normalize=True)
    model_final.fit(X[importantFeatures.feature],y)

    return model_final,importantFeatures

In [13]:
#save relative importance
model_CPC,importantFeatures_CPC=runRelativeImportance('CPC',NetreRun=False,modelreRun=False)
model_CTR,importantFeatures_CTR=runRelativeImportance('CTR',NetreRun=False,modelreRun=False)

CPC GBM r2 0.04546807371229611
CPC MLR r2 0.04972193179901718
CTR GBM r2 0.013501891466862537
CTR MLR r2 0.014650273482348708


* In general models capacity to predict performance in both metric is weak, GMB is better than liner regression but in both cases the percentage of variability explained by the model is low 
* With this caviat, we can plot the results and get some insights that should be directionally correct

In [87]:
importantFeatures_CPC.to_excel('Results_Importance_CPC.xlsx',index=False)
importantFeatures_CTR.to_excel('Results_Importance_CTR.xlsx',index=False)

In [24]:
values=model_CPC.coef_
tag_type=[element.split('|')[0] for element in model_CPC.feature_names_in_]
tag_name=[element.split('|')[1] for element in model_CPC.feature_names_in_]

results=pd.DataFrame(columns=['Tag_type','Tag_name','Value'],index=range(len(values)))

results['Tag_type']=tag_type
results['Tag_name']=tag_name
results['Value']=values
# results['Metric']='CPC'
cpc=results.sort_values('Value',ascending=False)

In [25]:
values=model_CTR.coef_
tag_type=[element.split('|')[0] for element in model_CTR.feature_names_in_]
tag_name=[element.split('|')[1] for element in model_CTR.feature_names_in_]

results=pd.DataFrame(columns=['Tag_type','Tag_name','Value'],index=range(len(values)))

results['Tag_type']=tag_type
results['Tag_name']=tag_name
results['Value']=values
# results['Metric']='CTR'
ctr=results.sort_values('Value',ascending=False)

###Determine frecuency for relevant features

In [72]:
#take creative element important for both metrics
result=pd.merge(cpc,
                ctr,
                how="inner",
                on=['Tag_type','Tag_name'],
                suffixes=("_cpc", "_ctr"),
                copy=True,
                indicator=False,
                validate=None)


In [77]:
#add asset type and exposition time 

aux=orignalData.groupby(['Tag_type','Tag_name'])['Frecuency'].sum().reset_index()
result=pd.merge(result,
                aux,
                how="left",
                on=['Tag_type','Tag_name'],
                suffixes=("_r", "_r2"),
                copy=True,
                indicator=False,
                validate=None)

In [81]:
#normalize results 
result['Value_cpc']=MinMaxScaler((-1,1)).fit_transform(np.array(result['Value_cpc']).reshape(-1,1))
result['Value_ctr']=MinMaxScaler((-1,1)).fit_transform(np.array(result['Value_ctr']).reshape(-1,1))
result['Frecuency']=MinMaxScaler((0,1)).fit_transform(np.array(result['Frecuency']).reshape(-1,1))

In [83]:
result.to_excel('results.xlsx', index=False)